In [1]:

# 用于rules中删除通配符规则算法
#
from functools import  reduce
from copy import copy, deepcopy
# a = {'com': {'apple': {'push': {'36c253f27fe20fa45bc60f58': 'cn'}}}}
# a = {'com': {'apple': {'push': {'36c253f27fe20fa45bc60f58': 'cn'}}, 'baidu': {'www': {'36c253f27fe20fa45bc60f58': 'cn'}}}}
# a = {'com': {'apple': {'push': {'36c253f27fe20fa45bc60f58': 'cn'}, 'www': {'36c253f27fe20fa45bc60f58': 'cn'}}}}
# a = {'com': {'apple': {'push': {'36c253f27fe20fa45bc60f58': 'cn'}, 'www': {'36c253f27fe20fa45bc60f58': 'cn'}}}, 'cn': {'baidu': {'push': {'ad': {'36c253f27fe20fa45bc60f58': 'cn'}}}}}
# a = {'com': {'apple': {'push': {'ad': {'36c253f27fe20fa45bc60f58': 'cn'}}, 'www': {'36c253f27fe20fa45bc60f58': 'cn'}}}}
# a = {'com': {'apple': {'push': {'ad': {'36c253f27fe20fa45bc60f58': 'cn'}, '36c253f27fe20fa45bc60f58': 'cn'}, 'www': {'36c253f27fe20fa45bc60f58': 'cn'}}}, 'baidu': {'push': {'ad': {'36c253f27fe20fa45bc60f58': 'cn'}}}}

a = {'net': {'love67': {'dfcb0ab9a701050fc224fcd7': 'proxy'}}, 'com': {'baidu': {'ad': {'dfcb0ab9a701050fc224fcd7': 'cn'}}}}
a_del = ['com', 'baidu', 'ad']

def recder_decorator(function):
    """
    函数作用： 该装饰器用于删除通配符规则时的记录路径
    变量：
        updata作用： 用于记录需要删除整个字典的路径，将获取到的路径插入到列表中
        upkey作用： 用于记录需要删除整个字典的key，将获取到的路径插入到列表中

    注意：由于default_rule使用了随机值，因此不可使用configs中的default_rule
    """
    updata = []
    upkey = []
    default_rule = None
    def wrapper(*args, **kwargs):
        nonlocal updata, upkey, default_rule
        updata.insert(0, args[0])
        upkey.insert(0, args[1])
        _end, default_rule = function(args[0], args[1], updata=updata, upkey=upkey, default_rule=default_rule, data=args[2])
        return _end
    return wrapper

@recder_decorator
def delete_validated(x, y, *, updata: list[dict], upkey: list[str], default_rule: str = None, data: list = None):
    """
    遍历通配符rules缓存，验证是否有可删除的数据, 并记录查找到的default_rule
    因为default_rule随机字符串，每次重启服务后变化
    如果未查询到default_rule，则不删除rules缓存操作，因为查询时如果未查询到，则会返回None
    """
    if isinstance(x, dict):
        _end = x.get(y)
        if _end:
            if len(_end) == 1:
                """
                _end: 中只有一个数据
                example: 
                    {'ad': {'36c253f27fe20fa45bc60f58': 'cn'}}
                """
                if len(set(data) - set(upkey)) == 0 and isinstance(*_end.values(), str):
                    """
                    判断需要删除域名列表长度与装饰器中的列表长度是否一致，以此为判断遍历结束，
                    且数据为{'ad': {'36c253f27fe20fa45bc60f58': 'cn'}}时
                    且需要删除的域名列表长度为1，就说明遍历结束，保存key,并执行删除方法
                    """
                    default_rule = [*_end.keys()][0]
                    updata.insert(0, _end)
                    upkey.insert(0, default_rule)
                    delete_vaule(real_updata=updata, real_upkey=upkey, default_rule=default_rule)
                    return _end, default_rule
                else:
                    return _end, default_rule
            else:
                if len(set(data) - set(upkey)) == 0:
                    """
                    用于判断遍历是否结束
                    如果结束，但是_end中的值为：{'ad': {'36c253f27fe20fa45bc60f58': 'cn'}, '36c253f27fe20fa45bc60f58': 'cn'}
                    也就说明需要删除的通配符域名下有子域或主机单独设置了规则
                    设计思路：不删除通配符以下的主机规则，只删除通配符当前层级规则，因为主机优先，未匹配到主机才选择选择规则
                    """
                    # 判断需要遍历的数据已经完成
                    for k, v in _end.items():
                        if isinstance(v, str):
                            default_rule = k
                    delete_vaule(real_updata=updata, real_upkey=upkey, default_rule=default_rule)
                return _end, default_rule
        else:
            return _end, default_rule
    else:
        return x, None

def delete_vaule(real_updata, real_upkey, default_rule):
    """
    删除通配符规则方法
    for 循环下使用深度复制，一个用于遍历，一个真实变量用于操作修改
    """
    _upkey = deepcopy(real_upkey)
    _updata = deepcopy(real_updata)
    for key, value in zip(_upkey, _updata):
        _data = value.get(key)
        if default_rule in _data:
            # 判断是否有默认rules
            real_data = real_updata[_updata.index(value)]
            if len(_data) != 1:
                real_data[key].pop(default_rule)
            else:
                real_data.pop(key)
                if len(real_data) != 0:
                    # 判断pop后是否还有数据, 如果没有数据继续循环一次将空字典pop
                    break
        else:
            # 验证configs in _data操作后是否没有任何键值对
            if len(real_updata[_updata.index(value)].get(key)) == 0:
                real_updata[_updata.index(value)].pop(key)
            continue

print(a)
reduce(lambda x, y: delete_validated(x, y, a_del), a_del, a)
print(a)

{'net': {'love67': {'dfcb0ab9a701050fc224fcd7': 'proxy'}}, 'com': {'baidu': {'ad': {'dfcb0ab9a701050fc224fcd7': 'cn'}}}}
{'net': {'love67': {'dfcb0ab9a701050fc224fcd7': 'proxy'}}}


In [ ]:

# 使用reduce将list中的域名转换为dict，作为rules查询基础使用
from functools import  reduce

rulescache = {}


i_list = ['ad', 'abc', 'www', 'baidu', 'com']
i_list.reverse()
# l_list = ['abc', 'beiba', 'baidu', 'com']
l_list = ['baidu', 'com']
l_list2 = ['www', 'aac', 'baidu', 'com']
# l_list2 = ['gggggggggggg', 'ffffffff', 'aac', 'www', 'baidu', 'com']
# l_list = ['baidu', 'com']
l_list.reverse()
l_list2.reverse()
g_list = ['www', 'google', 'com']
g_list2 = ['www', 'google', 'cn']
g_list.reverse()
g_list2.reverse()

error_list = ['cn']

dn_list = []
dn_list.append(g_list)
dn_list.append(error_list)
# dn_list.append(i_list)
# dn_list.append(l_list)
# dn_list.append(g_list2)
dn_list.append(l_list2)

def cache_search(fun):
    rulescache2 = {}
    def wapper(*args, **kwargs):
        print(*args)
        nonlocal rulescache2
        rulescache2 |= args[0]
        return fun(*args)
    return wapper

for i in dn_list:
    def convertdict(updict, nextkey, datalist):
        if isinstance(updict, str):
            if len(datalist) > 2:
                one_key = rulescache.get(updict)
                if one_key is None:
                        rulescache.update({updict: {}})
                        two_key = rulescache.get(updict)
                        two_key.update({nextkey: {}})
                        return two_key.get(nextkey)
                else:
                    savedata = one_key.get(nextkey)
                    if savedata:
                        return savedata
                    else:
                        one_key.update({nextkey: {}})
                        two_key = one_key.get(nextkey)
                        return two_key
            else:
                one_key = rulescache.get(updict)
                if one_key is None:
                        rulescache.update({updict: {}})
                        two_key = rulescache.get(updict)
                        two_key.update({nextkey: {'default-rule': 'cn'}})
                        return two_key.get(nextkey)
                else:
                    savedata = one_key.get(nextkey)
                    if savedata:
                        savedata.update({'default-rule': 'cn'})
                        return savedata
                    else:
                        one_key.update({nextkey: {'default-rule': 'cn'}})
                        two_key = one_key.get(nextkey)
                        return two_key
                
        elif isinstance(updict, dict):
            if nextkey == datalist[-1]:
                # 判断是否为最后一级域名
                # 与
                # 只有顶级域
                if len(updict) == 0:
                    updict.update({nextkey: {'default-rule': 'cn'}})
                else:
                    top_key = updict.get(nextkey)
                    # 检查传入的字典缓存是否已经存在顶级域
                    if top_key:
                        top_key.update({'default-rule': 'cn'})
                    else:
                        updict[nextkey] = {'default-rule': 'cn'}
            else:
                if len(updict) == 0:
                    updict.update({nextkey: {}})
                else:
                    _tmp = updict.get(nextkey)
                    if _tmp:
                        return _tmp
                    else:
                        updict.update({nextkey: {}})
                        _tmp = updict.get(nextkey)
                        return _tmp
            return updict.get(nextkey)
    reduce(lambda x,y: convertdict(x, y, i), i, rulescache)

print(rulescache.get('com').get('baidu').get('default-rule'))
print(rulescache)